In [50]:
# 초반 : 레이블링을 통해 추천
# 1. 서비스 시작 시, 카테고리와 필터 분포가 다양하게 n개의 플레이스를 보여주면서 마음에 드는 장소를 선택하라고 함
# 2. 사용자가 선택한 플레이스의 카테고리와 필터를 바탕으로 비슷한 장소를 n개 추천
# 3. 찜하기 테이블이 어느정도 채워지면, 레이블링과 UBCF+IBCF를 모두 사용하여 추천해주는 방식으로 설계

In [51]:
import numpy as np
import pandas as pd

In [52]:
places = pd.read_csv(f"C:/Users/김가연/Desktop/23-2학기/캡스톤2/data/완료/places.csv", encoding='UTF-8-SIG')
filters_places = pd.read_csv(f"C:/Users/김가연/Desktop/23-2학기/캡스톤2/data/완료/filters_places.csv", encoding='UTF-8-SIG')

In [53]:
places.head()

,id,category_id,name,district,address,latitude,longitude,work_time,price,image_url,page_url,description,phone_number
0,1,3,쉘터오브메모리즈,용산구,"서울시 용산구 용산동6가 11-376, 국립중앙박물관 입구 건너편 정류장",37.520797,126.978958,연중무휴,무료,https://news.seoul.go.kr/culture/files/2016/12...,https://design.seoul.go.kr/gallery/main/main.jsp,NaN,NaN
1,2,3,쉘터오브메모리즈,용산구,"서울시 용산구 용산동6가 11-375, 국립중앙박물관 입구 정류장",37.520707,126.979676,연중무휴,무료,https://news.seoul.go.kr/culture/files/2016/12...,https://design.seoul.go.kr/gallery/main/main.jsp,NaN,NaN
2,3,3,남산의 생태,용산구,"서울시 용산구 후암동 406-98, 남산소월길, 후암약수터 버스정류장",37.548463,126.984184,연중무휴,무료,https://publicart.eseoul.go.kr/common/file/get...,https://design.seoul.go.kr/gallery/main/main.jsp,NaN,NaN
3,4,3,회화적 몽타주,용산구,"서울시 용산구 후암동 30-84, 남산소월길, 남산도서관 앞 버스정류장",37.552732,126.981128,연중무휴,무료,https://news.seoul.go.kr/culture/files/2016/12...,https://design.seoul.go.kr/gallery/main/main.jsp,NaN,NaN
4,5,3,휴식,용산구,"서울시 용산구 용산동2가 1-1144, 남산소월길, 보성여중고 버스정류장",37.546340,126.985610,연중무휴,무료,https://publicart.eseoul.go.kr/common/file/get...,https://design.seoul.go.kr/gallery/main/main.jsp,NaN,NaN


In [54]:
filters_places.head()

,id,place_id,name,filter_id
0,1,1,쉘터오브메모리즈,3
1,2,1,쉘터오브메모리즈,6
2,3,2,쉘터오브메모리즈,3
3,4,2,쉘터오브메모리즈,6
4,5,3,남산의 생태,3


In [55]:
merged_df = pd.merge(places, filters_places, left_on='id', right_on='place_id')
merged_df.head()

,id_x,category_id,name_x,district,address,latitude,longitude,work_time,price,image_url,page_url,description,phone_number,id_y,place_id,name_y,filter_id
0,1,3,쉘터오브메모리즈,용산구,"서울시 용산구 용산동6가 11-376, 국립중앙박물관 입구 건너편 정류장",37.520797,126.978958,연중무휴,무료,https://news.seoul.go.kr/culture/files/2016/12...,https://design.seoul.go.kr/gallery/main/main.jsp,NaN,NaN,1,1,쉘터오브메모리즈,3
1,1,3,쉘터오브메모리즈,용산구,"서울시 용산구 용산동6가 11-376, 국립중앙박물관 입구 건너편 정류장",37.520797,126.978958,연중무휴,무료,https://news.seoul.go.kr/culture/files/2016/12...,https://design.seoul.go.kr/gallery/main/main.jsp,NaN,NaN,2,1,쉘터오브메모리즈,6
2,2,3,쉘터오브메모리즈,용산구,"서울시 용산구 용산동6가 11-375, 국립중앙박물관 입구 정류장",37.520707,126.979676,연중무휴,무료,https://news.seoul.go.kr/culture/files/2016/12...,https://design.seoul.go.kr/gallery/main/main.jsp,NaN,NaN,3,2,쉘터오브메모리즈,3
3,2,3,쉘터오브메모리즈,용산구,"서울시 용산구 용산동6가 11-375, 국립중앙박물관 입구 정류장",37.520707,126.979676,연중무휴,무료,https://news.seoul.go.kr/culture/files/2016/12...,https://design.seoul.go.kr/gallery/main/main.jsp,NaN,NaN,4,2,쉘터오브메모리즈,6
4,3,3,남산의 생태,용산구,"서울시 용산구 후암동 406-98, 남산소월길, 후암약수터 버스정류장",37.548463,126.984184,연중무휴,무료,https://publicart.eseoul.go.kr/common/file/get...,https://design.seoul.go.kr/gallery/main/main.jsp,NaN,NaN,5,3,남산의 생태,3


In [56]:
# 필요한 열만 선택
place = merged_df[['category_id', 'filter_id','name_x', 'id_x']]

place = place.rename(columns={'name_x' : 'name', 'id_x' : 'id'})
place

,category_id,filter_id,name,id
0,3,3,쉘터오브메모리즈,1
1,3,6,쉘터오브메모리즈,1
2,3,3,쉘터오브메모리즈,2
3,3,6,쉘터오브메모리즈,2
4,3,3,남산의 생태,3
...,...,...,...,...
4543,1,4,장충테니스장,1899
4544,1,5,장충테니스장,1899
4545,1,1,장충리틀야구장,1900
4546,1,4,장충리틀야구장,1900


In [57]:
# category_id가 1에서 7까지인 행만 선택
selected_rows = place[place['category_id'].between(1, 7)]

In [58]:
# 각 카테고리별로 랜덤하게 1개씩 선택
place_sampled = selected_rows.groupby(['category_id', 'filter_id']).apply(lambda x: x.sample(n=min(1, len(x)))).reset_index(drop=True)
place_sampled

,category_id,filter_id,name,id
0,1,1,배드민턴(우장산 배드민턴장),157
1,1,2,"스트레칭, 헬스(도림동자치센터 헬스장)",168
2,1,3,"스트레칭,헬스(도림동자치센터 헬스장)",48
3,1,4,라인댄스(주민센터3층체육관),110
4,1,5,장충리틀야구장,1900
5,1,6,서울시청 하늘광장 갤러리,1468
6,2,1,2023 매헌시민의숲 가을축제 안내,710
7,2,2,연우소극장,1354
8,2,3,파리공원 야외공연장,1529
9,2,4,[2023 성동문화마당] 미술관으로 떠나는 해외 여행 콘서트,689


In [59]:
# 이 랜덤으로 뽑힌 플레이스에서 사용자가 찜 버튼을 누르면 해당 플레이스의 찜이 저장됨
# 저장됨 찜을 바탕으로 플레이스의 카테고리, 필터가 동일한 장소들을 추천해주는 방식

In [60]:
#np.random.seed(42)

# 임의로 사용자가 찜한 플레이스 목록 생성
user_likes = pd.DataFrame(place_sampled.sample(n=3))
user_likes


,category_id,filter_id,name,id
33,6,7,독다방(구 독수리다방),1781
15,3,5,구로공단 노동자생활 체험관 (순이의 집),1875
19,4,4,카모메그림책방,565


In [61]:
user_liked_places = pd.merge(user_likes, place, how='left')
user_liked_places

,category_id,filter_id,name,id
0,6,7,독다방(구 독수리다방),1781
1,3,5,구로공단 노동자생활 체험관 (순이의 집),1875
2,4,4,카모메그림책방,565


In [62]:
# 사용자가 찜한 플레이스와 동일한 카테고리, 필터를 가진 다른 플레이스 추천
def recommend_similar_places(user_liked_places, place, target_count):
    recommended_places = pd.DataFrame(columns=place.columns)
    
    for index, row in user_liked_places.iterrows():
        similar_places = place[(place['category_id'] == row['category_id']) & (place['filter_id'] == row['filter_id'])]
        similar_places = similar_places[similar_places['name'] != row['name']]
        
        if not similar_places.empty:
            sampled_places = similar_places.drop_duplicates(subset=['name'])
            sampled_places = sampled_places.sample(n=min(target_count, len(similar_places)), replace=True) # 각 조건에 해당하는 장소를 n개씩 뽑음
            recommended_places = pd.concat([recommended_places, sampled_places], ignore_index=True)
            recommended_places = recommended_places.drop_duplicates(subset=['name'])
            recommended_places = recommended_places.sample(frac=1, random_state=42).head(5) # 카테고리, 필터가 일치하는 장소 중에 총 5개만 뽑음
    
    return recommended_places

In [63]:
# 사용자에게 추천할 플레이스
recommended_places = recommend_similar_places(user_liked_places, place, target_count=2)
recommended_places

,category_id,filter_id,name,id
0,6,7,라 칸티나,1761
1,3,5,환구단,1882
5,4,4,알라딘 중고매장(서울대입구역점),447
2,6,7,카페 귀천,1810
4,4,4,용산 고래이야기 작은도서관,1243


In [154]:
# train, test 데이터 분리
#from sklearn.model_selection import train_test_split)

In [155]:
# 유사도 계산 타니모토 계수

In [ ]:
# 성과 측정 지표 : 정밀도, 재현율 동시에 고려하는 조화평균(F1 score) 사용
# (2x정밀도x재현율)/(정밀도+재현율)